# North Macedonian 2024 Election Fact Checking Pipeline
---
This notebook is part of the North Macedonian 2024 Election Misinformation Assessment project. It contains code to analyze and visualize misinformation related to the elections in North Macedonia.

##### Libraries

In [18]:
from dotenv import load_dotenv
import requests
import json
from urllib.parse import quote_plus
import tqdm
import json
import os
import csv
import pandas as pd
import time

##### Constants

In [5]:
load_dotenv()
llama3_url = "http://localhost:11434/api/generate"
google_api = os.getenv("GOOGLE-API-KEY")
cx = os.getenv("CX")

dataset = "Dataset/VMRO-DPMNE_ad_creative_bodies.csv"
party = "VMRO-DPMNE"

## Data Collection

In [7]:
pandas_df = pd.read_csv(dataset)
print(pandas_df.head())

      ad_archive_id                                 ad_creative_bodies
0   427480589881211  Para disa ditëve në emisionin “Politiko” e kam...
1  1636678057070325  Промените се во твои раце!\nМакедонија повторн...
2   252100357930225                        За многу години Цветници!!!
3  1117244869521215  ✒️Shkruan: @[100005900348509:2048:Enis Shaqiri...
4  1595858924569626  ИЗГУБЕНА Е ДОВЕРБАТА ВО ПОЛИТИЧАРИТЕ БИДЕЈЌИ Н...


In [6]:
parties = ["DUI", "SDSM", "Left", "VMRO-DPMNE"]
parties_csv_name = ["DUI.csv", "meta-ad-library-2025-04-30_2-SDSM.csv", "meta-ad-library-2025-04-30_3-Левица.csv", "meta-ad-library-2025-04-30-VMRO-DPMNE.csv"]
parties_csv_name_dict = {
    "DUI": "DUI.csv",
    "SDSM": "meta-ad-library-2025-04-30_2-SDSM.csv",
    "Left": "meta-ad-library-2025-04-30_3-Левица.csv",
    "VMRO-DPMNE": "meta-ad-library-2025-04-30-VMRO-DPMNE.csv"
}

## Data Preprocessing

### Extract Claims

In [24]:

def extract_claims(text, id, iteration):
    promptQuestion= """For the bellow text, extract at max 3 claims and give to me in the JSON format. The claims should in English Language. You should only return the json. Do not add any other text. The JSON should only have 1 key "Claims" and the value should be a list of claims."""
    
    extract_claim_payload = {
    "model": "llama3",
    "prompt": promptQuestion + text,
    }
    
    extract_claim_response = requests.post(llama3_url, json=extract_claim_payload)
    
    if extract_claim_response.status_code == 200:
        lines = extract_claim_response.text.splitlines()
        output = ""
        for line in lines:
            try:
                # Parse each line as JSON
                line_data = json.loads(line)
                # Concatenate the "response" field
                output += line_data.get("response", "")
            except json.JSONDecodeError:
                # Skip lines that are not valid JSON
                continue
        # Parse the final concatenated output as JSON
        try:
            final_output = json.loads(output)
            
            print("Extracted Claims: Iteration:", iteration, "ID:", id, "Claims:", final_output)
            return final_output
        except json.JSONDecodeError:
            # Add a } to the end of the output
            output += "}"
            try:
                final_output = json.loads(output)
                print("Extracted Claims: Iteration:", iteration, "ID:", id, "Claims:", final_output)
                return final_output
            except json.JSONDecodeError:
                print("\n")
                print("Final output is not valid JSON: Iteration:", iteration, "ID:", id, "Output:", output)
                print("\n")
                return None
    else:
        print(f"Error: {extract_claim_response.status_code}")
        print(extract_claim_response.text)

In [19]:
ids_to_claims = {}
read_csv_path_bodies = "./dataset/ad-creative-bodies-extraction/"

for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(read_csv_path_bodies, party)
    csv_file_path = csv_file_path + "_ad_creative_bodies.csv"
    # Add "ad_creative_bodies" in front of the file name
    # csv_file_path = csv_file_path.replace(".csv", "_ad_creative_bodies.csv")
    print(csv_file_path)
    pandas_df = pd.read_csv(csv_file_path)
    
    # Print the DataFrame shape
    print(f"Shape of {party} DataFrame:", pandas_df.shape)
    
    # Check if the DataFrame is empty
    if pandas_df.empty:
        print(f"The DataFrame for {party} is empty.")
        continue
    
    # Iterate through the rows of the DataFrame


# for index, row in tqdm.tqdm(pandas_df.iterrows(), total=pandas_df.shape[0]):
#     # Extract the text from the DataFrame
#     text = row['ad_creative_bodies']
#     id = row['ad_archive_id']
#     # Extract claims from the text
#     claims = extract_claims(text)
#     # Store the claims in the dictionary
#     ids_to_claims[id] = claims
    
# # Check if the folder and file exist
# if not os.path.exists("./PreprocessedDataset/"):
#     os.makedirs("./PreprocessedDataset/")
    

# # Save the claims to a CSV file
# with open("./PreprocessedDataset/" + party + "_claims.csv", "w", newline='', encoding='utf-8') as csvfile:
#     fieldnames = ['ad_archive_id', 'claims']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     for id, claims in ids_to_claims.items():
#         writer.writerow({'ad_archive_id': id, 'claims': json.dumps(claims)})

./dataset/ad-creative-bodies-extraction/DUI_ad_creative_bodies.csv
Shape of DUI DataFrame: (102, 2)
./dataset/ad-creative-bodies-extraction/SDSM_ad_creative_bodies.csv
Shape of SDSM DataFrame: (60, 2)
./dataset/ad-creative-bodies-extraction/Left_ad_creative_bodies.csv
Shape of Left DataFrame: (86, 2)
./dataset/ad-creative-bodies-extraction/VMRO-DPMNE_ad_creative_bodies.csv
Shape of VMRO-DPMNE DataFrame: (28, 2)
./dataset/ad-creative-bodies-extraction/ZNAM_ad_creative_bodies.csv
Shape of ZNAM DataFrame: (1, 2)


In [26]:
save_folder_path = "./dataset/claims-extraction/"

# Extract the Claims through LLM
for party in tqdm.tqdm(parties):
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(read_csv_path_bodies, party)
    csv_file_path = csv_file_path + "_ad_creative_bodies.csv"
    # Add "ad_creative_bodies" in front of the file name
    # csv_file_path = csv_file_path.replace(".csv", "_ad_creative_bodies.csv")
    print(csv_file_path)
    pandas_df = pd.read_csv(csv_file_path)
    
    # Print the DataFrame shape
    print(f"Shape of {party} DataFrame:", pandas_df.shape)
    
    # Check if the DataFrame is empty
    if pandas_df.empty:
        print(f"The DataFrame for {party} is empty.")
        continue
    
    iteration = 1
    # Iterate through the rows of the DataFrame
    for index, row in pandas_df.iterrows():
        # Extract the text from the DataFrame
        text = row['ad_creative_bodies']
        text = str(text)
        
        id = row['ad_archive_id']
        id = str(id)
        
        # Extract claims from the text
        claims = extract_claims(text, id, iteration)
        # Store the claims in the dictionary
        ids_to_claims[id] = claims
        iteration += 1
    # Save the claims to a CSV file
    with open(os.path.join(save_folder_path, party + "_claims.csv"), "w", newline='', encoding='utf-8') as csvfile:
        fieldnames = ['ad_archive_id', 'claims']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for id, claims in ids_to_claims.items():
            writer.writerow({'ad_archive_id': id, 'claims': json.dumps(claims)})
    ids_to_claims = {}
    

  0%|          | 0/5 [00:00<?, ?it/s]

./dataset/ad-creative-bodies-extraction/DUI_ad_creative_bodies.csv
Shape of DUI DataFrame: (102, 2)
Extracted Claims: Iteration: 1 ID: 6795509300551996 Claims: {'Claims': ['DUI will go into opposition', 'DUI will respond to crime and corruption']}
Extracted Claims: Iteration: 2 ID: 746708557578780 Claims: {'Claims': ['Three platforms are available for the upcoming parliamentary elections: Tirana platform, Sofia platform, and Macedonian platform.', 'The left will coalition with a government that accepts red lines in defending the Macedonian state.', 'Macedonians will put an end to the dictatorships and defeats of the SDS and DUI regime and restore the Republic.']}
Extracted Claims: Iteration: 3 ID: 464195942757638 Claims: {'Claims': ['The movement ZNAK for Our Macedonia is clear in its stance that it is against the opening of mines in Strumica.', "It should not be allowed to trade with people's health.", 'There needs to be an investigation whether DUI functionaries traded influence.']}


 20%|██        | 1/5 [02:33<10:13, 153.27s/it]

Extracted Claims: Iteration: 102 ID: 1585098798694924 Claims: {'Claims': ['Demokrati claims that the calculations of narco-bands indicate a brutal election.', 'Different political parties and their governing bodies over the past three decades have created and nurtured dangerous criminal bands which now take revenge on their own.', 'The spiral of violence must be stopped because it is consuming us.']}
./dataset/ad-creative-bodies-extraction/SDSM_ad_creative_bodies.csv
Shape of SDSM DataFrame: (60, 2)
Extracted Claims: Iteration: 1 ID: 988735132879405 Claims: {'Claims': ['There is no giving up on destiny', 'SDSM', 'Aerodrome', 'Macedonia']}
Extracted Claims: Iteration: 2 ID: 1138731707263665 Claims: {'Claims': ['SVM is the only party prepared to fight for citizens and the future of the state.', 'We are united and strong, we have endured time and weather, good and bad, and always been on the guard of the homeland.', "It's not an option for us to give up. We're a party made of solid materi

 40%|████      | 2/5 [03:57<05:38, 112.81s/it]

...

Extracted Claims: Iteration: 86 ID: 3593778017563269 Claims: ['NGOs are pijavici (scammers) who circulate public funds to spread Western propaganda', 'The City of Skopje, specifically the Sector for International Cooperation, should have a special program for cooperation with NGOs and foundations', 'It is not enough that NGOs are involved in the implementation of every program and activity of the City of Skopje']
./dataset/ad-creative-bodies-extraction/VMRO-DPMNE_ad_creative_bodies.csv
Shape of VMRO-DPMNE DataFrame: (28, 2)
Extracted Claims: Iteration: 1 ID: 427480589881211 Claims: {'Claims': ['The anti-Albanian quarters have a tendency to abolish amnesty and obscure the fight for freedom and justice of UÇK.', "Gordana Siljanovska's words confirmed that VMRO-DPMNE's political battle is to subjugate Albanians, not continue the partnership built since 2001", "EU Front should gain more than 22 mandates and 'Siljanovskas' should remain only a memory"]}
Extracted Claims: Iteration: 2 ID: 16

 80%|████████  | 4/5 [06:31<01:23, 83.55s/it] 

Extracted Claims: Iteration: 28 ID: 369290982662369 Claims: [{'Claims': ['President Stevo Pendarovski will run for a second term with support from LSDM.', 'Gordana Siljanovska Davkova, a deputy from VMRO-DPMNE, will also run again after five years.', 'Arben Taravari has been officially promoted as a presidential candidate by the VLEN coalition.']}]
./dataset/ad-creative-bodies-extraction/ZNAM_ad_creative_bodies.csv
Shape of ZNAM DataFrame: (1, 2)


100%|██████████| 5/5 [06:32<00:00, 78.47s/it]



Final output is not valid JSON: Iteration: 1 ID: 7582091031857061 Output: [
"Claims": [
"What attributes should the next President of North Macedonia possess?",
"The candidates have started to offer a wide range of promises and slogans, all with the aim of attracting voters.",
"There is no concrete answer as to what these candidates are offering."
]
]}




In [104]:
# Remove the columns which have none in the claims
for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(save_folder_path, party + "_claims.csv")
    pandas_df = pd.read_csv(csv_file_path)
    print("Before removing the columns for the Party " + party + " the shape is: ", pandas_df.shape)
    
    
    # Check if the DataFrame is empty
    if pandas_df.empty:
        print(f"The DataFrame for {party} is empty.")
        continue
    
    # Remove the rows where claims are None
    pandas_df = pandas_df[pandas_df['claims'].notna()]

    print("After removing the columns for the Party " + party + " the shape is: ", pandas_df.shape)
    print("\n")
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_csv_file_path = os.path.join(save_folder_path, party + "_claims_cleaned.csv")
    pandas_df.to_csv(cleaned_csv_file_path, index=False)

Before removing the columns for the Party DUI the shape is:  (148, 2)
After removing the columns for the Party DUI the shape is:  (135, 2)


Before removing the columns for the Party SDSM the shape is:  (60, 2)
After removing the columns for the Party SDSM the shape is:  (58, 2)


Before removing the columns for the Party Left the shape is:  (86, 2)
After removing the columns for the Party Left the shape is:  (82, 2)


Before removing the columns for the Party VMRO-DPMNE the shape is:  (28, 2)
After removing the columns for the Party VMRO-DPMNE the shape is:  (26, 2)


Before removing the columns for the Party ZNAM the shape is:  (1, 2)
After removing the columns for the Party ZNAM the shape is:  (0, 2)




### Claim to Questions

In [36]:
def claim_to_question(claim):
    prompt_question = f""" For the bellow claim generate me 3 neural fact-checking questions and give to me in the format of a json. You should only return the json. The json should have only 1 key `Questions` and the value should be a list of questions.
    Claim: "{claim}"""
    
    extract_claim_payload = {
    "model": "llama3",
    "prompt": prompt_question + text,
    }
    
    question_response = requests.post(llama3_url, json=extract_claim_payload, timeout=30)
    
    if question_response.status_code == 200:
        # Process the streamed response
        lines = question_response.text.splitlines()
        output = ""

        for line in lines:
            try:
                # Parse each line as JSON
                line_data = json.loads(line)
                # Concatenate the "response" field
                output += line_data.get("response", "")
            except json.JSONDecodeError:
                # Skip lines that are not valid JSON
                continue

        # Parse the final concatenated output as JSON
        try:
            final_output = json.loads(output)
            return final_output
        except json.JSONDecodeError:
            # Add a } to the end of the output
            output += "}"
            try:
                final_output = json.loads(output)
                return final_output
            except json.JSONDecodeError:
                return None
    else:
        print(f"Error: {question_response.status_code}")
        print(question_response.text)

In [105]:
# Open the Claims and read everything once to make sure that the json is properly formated
for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(save_folder_path, party + "_claims_cleaned.csv")
    pandas_df = pd.read_csv(csv_file_path)
    print(pandas_df.head())
    
    
    # # Iterate through the rows of the DataFrame
    # for index, row in pandas_df.iterrows():
    #     # Extract the text from the DataFrame
    #     claims = row['claims']
    #     claims = json.loads(claims)
    #     print(claims)
    
    # Iterate through the claims. If the claim is found which is an array, add it to the dictionary
    for index, row in pandas_df.iterrows():
        # Extract the text from the DataFrame

        try:
            claims = row['claims']
            claims = json.loads(claims)
            c = claims["Claims"]
        except TypeError:
            
            if len(claims) == 1:
                print("Array Error")
                # Update the dataframe
                pandas_df.at[index, 'claims'] = claims[0]
                print(pandas_df.at[index, 'claims'])
            
            else:
                
                # Add the claim to the dictionary
                print("No Claims field error")
                # Update the dataframe
                claims = {"Claims": claims}
                pandas_df.at[index, 'claims'] = claims

        
        
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_csv_file_path = os.path.join(save_folder_path, party + "_claims_cleaned.csv")
    pandas_df.to_csv(cleaned_csv_file_path, index=False)
    


      ad_archive_id                                             claims
0   988735132879405  {"Claims": ["\u041d\u0435\u043c\u0430\u041e\u0...
1  1138731707263665  {"Claims": ["SDSM is the only party ready to f...
2  1462600661315120  {"Claims": ["SOSM is the only party ready to f...
3  1168468637617703  {"Claims": ["It is not too late to change dire...
4  1366791877252741  {"Claims": ["We are striving for great support...
No Claims field error
Array Error
{'Claims': ['The public-private partnership agreement for the Smart Airport project worth 10.7 million euros was signed on March 25, 2024.', 'There is no mention of digitization of municipal administrative services in the agreement.', 'The agreement does not specify how it will stimulate innovation and startup businesses.']}
No Claims field error
No Claims field error
Array Error
{'Claims': ['The government led by SDSM has separated 700,000 denari for the citizens of Aerodrom municipality to work on technical documentation for buildin

In [111]:
# Read the Claims and each claim and if there is an error remove the row

for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(save_folder_path, party + "_claims_cleaned.csv")
    pandas_df = pd.read_csv(csv_file_path)
    print("Shape Before Filtering: ", pandas_df.shape)
    
    # Iterate through the rows of the DataFrame
    for index, row in pandas_df.iterrows():
        # Extract the text from the DataFrame
        claims = row['claims']
        try:
            claims = json.loads(claims)
            c = claims["Claims"]
        except:
            # Remove the row from the DataFrame
            pandas_df = pandas_df.drop(index)
            print("Removed the row with index: ", index)
            continue
    
    print("Shape After Filtering: ", pandas_df.shape)
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_csv_file_path = os.path.join(save_folder_path, party + "_claims_cleaned.csv")
    pandas_df.to_csv(cleaned_csv_file_path, index=False)
    

Shape Before Filtering:  (135, 2)
Removed the row with index:  8
Removed the row with index:  10
Removed the row with index:  12
Removed the row with index:  28
Removed the row with index:  29
Removed the row with index:  30
Removed the row with index:  32
Removed the row with index:  46
Removed the row with index:  53
Removed the row with index:  54
Removed the row with index:  72
Removed the row with index:  77
Removed the row with index:  79
Removed the row with index:  92
Removed the row with index:  94
Removed the row with index:  97
Removed the row with index:  99
Removed the row with index:  105
Removed the row with index:  115
Removed the row with index:  123
Removed the row with index:  124
Removed the row with index:  125
Removed the row with index:  127
Shape After Filtering:  (112, 2)
Shape Before Filtering:  (58, 2)
Removed the row with index:  3
Removed the row with index:  12
Removed the row with index:  29
Removed the row with index:  33
Removed the row with index:  45


In [114]:
# Load the claims from the CSV file
# claims_df = pd.read_csv("./PreprocessedDataset/" + party + "_claims.csv")
# print(claims_df.head())

# ids_to_claims_to_questions = {}

# for index, row in tqdm.tqdm(claims_df.iterrows(), total=claims_df.shape[0]):
#     claims_to_questions = {}
#     # Extract the claims from the DataFrame
#     claims = json.loads(row['claims'])
#     id = row['ad_archive_id']
#     # Extract claims from the text
#     for claim in tqdm.tqdm(claims['Claims']):
#         questions = claim_to_question(claim)
#         # Store the claims in the dictionary
#         claims_to_questions[claim] = questions
#     # Store the claims in the dictionary
#     ids_to_claims_to_questions[id] = claims_to_questions
# # Check if the folder and file exist
# if not os.path.exists("./PreprocessedDataset/"):
#     os.makedirs("./PreprocessedDataset/")
# # Save the claims to a CSV file
# with open("./PreprocessedDataset/" + party + "_claims_to_questions.csv", "w", newline='', encoding='utf-8') as csvfile:
#     fieldnames = ['ad_archive_id', 'claims_to_questions']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     for id, claims in ids_to_claims_to_questions.items():
#         writer.writerow({'ad_archive_id': id, 'claims_to_questions': json.dumps(claims)})

saved_questions_path = "./dataset/questions-extraction/"
for party in tqdm.tqdm(parties):
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(save_folder_path, party + "_claims_cleaned.csv")
    pandas_df = pd.read_csv(csv_file_path)
    
    # Print the DataFrame shape
    print(f"Shape of {party} DataFrame:", pandas_df.shape)
    
    # Check if the DataFrame is empty
    if pandas_df.empty:
        print(f"The DataFrame for {party} is empty.")
        continue
    
    id_claims_to_questions = {}
    
    # Iterate through the rows of the DataFrame
    iteration = 1
    for index, row in tqdm.tqdm(pandas_df.iterrows(), total=pandas_df.shape[0]):
        print("Iteration: ", iteration)
        iteration += 1
        claims_to_questions = {}
        # Extract the claims from the DataFrame
        claims = row['claims']
        try:
            claims = json.loads(claims)
            c = claims["Claims"]
            id = row['ad_archive_id']
            # Extract claims from the text
            print(claims)
            for claim in claims['Claims']:
                questions = claim_to_question(claim)
                # Store the claims in the dictionary
                claims_to_questions[claim] = questions
            # Store the claims in the dictionary
            id_claims_to_questions[id] = claims_to_questions
            print(claims_to_questions)
        except:
            # Remove the row from the DataFrame
            pandas_df = pandas_df.drop(index)
            print("Removed the row with index: ", index)
            continue
        
        

    
    # Save the claims to a CSV file
    with open(os.path.join(saved_questions_path, party + "_claims_to_questions.csv"), "w", newline='', encoding='utf-8') as csvfile:
        fieldnames = ['ad_archive_id', 'claims_to_questions']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for id, claims in id_claims_to_questions.items():
            writer.writerow({'ad_archive_id': id, 'claims_to_questions': json.dumps(claims)})
        

  0%|          | 0/5 [00:00<?, ?it/s]

Shape of DUI DataFrame: (112, 2)


Iteration:  1
{'Claims': ['НемаОткажуваќьOdИднината', 'The SDSM is...', 'Macedonia']}


{'НемаОткажуваќьOdИднината': {'Questions': ['What are the key attributes that the next President of North Macedonia should possess?', 'How do the presidential candidates differ in their promises and slogans?', 'What specific policies or initiatives do the seven aspirants for the presidency promise to implement if elected?']}, 'The SDSM is...': {'Questions': ['What is the SDSM?', 'What are the attributes that the next President of North Macedonia should possess?', 'What promises and slogans do the presidential candidates offer to attract voters?']}, 'Macedonia': {'Questions': ['What is the current situation regarding the presidential election in North Macedonia?', 'Are there any specific requirements for the next president of North Macedonia, as mentioned in the claim?', "Who are the seven candidates competing for the presidency in North Macedonia's election?"]}}
Iteration:  2
{'Claims': ['SDSM is the only party ready to fight for citizens and the future of the state.', 'The party has e

{'SDSM is the only party ready to fight for citizens and the future of the state.': {'Questions': ["Is the claim 'SDSM is the only party ready to fight for citizens and the future of the state' supported by factual evidence?", 'What specific attributes should the next President of North Macedonia possess, according to the author Enis Shaqiri?', 'What are the concrete offers made by the presidential candidates Biljana Vankovski, Maksim Dimitrievski, and Stefçe Jakimovski?']}, 'The party has endured time and weather, good and bad, and has always been on guard for the homeland.': {'Questions': ['Is the party mentioned in the claim a political party?', 'What are the specific promises and slogans offered by the presidential candidates?', "What is the relevance of the phrase 'homeland' to the context of the presidential election?"]}, "It's not an option for us to give up; SDSM is a party made of solid material.": {'Questions': ['What is the context of the claim that SDSM is a party made of s

...

{'Hungary is the richest country in Europe': {'Questions': ['What is the source of the claim that Hungary is the richest country in Europe?', 'Who are the candidates running for President of North Macedonia and what promises do they offer?', 'What is the significance of the question about the attributes required by the next President of North Macedonia?']}, 'The Social Democratic Party of Macedonia (LSDM) and the Democratic Union for Integration (BDI) are going into opposition': {'Questions': ['What political parties are going into opposition in Macedonia?', 'What are the key differences between the presidential candidates in the Macedonian presidential election?', 'What is the current situation with the presidential election in North Macedonia and what are the main challenges for the future president?']}}
Iteration:  17
{'Claims': ['A new church is being built in Prevalec', 'New sports facilities and urban equipment are being installed in Prevalec', 'The area will be urbanized and dev

{'A new church is being built in Prevalec': {'Questions': ['Is it true that a new church is being built in Prevalec?', 'What specific attributes should the next President of North Macedonia possess, according to the claim?', 'Do the presidential candidates really offer a wide range of promises and slogans to attract voters?']}, 'New sports facilities and urban equipment are being installed in Prevalec': {'Questions': ['Is the claim that new sports facilities and urban equipment are being installed accurate?', 'What is the specific role or attribute required of the future President of North Macedonia, as mentioned in the claim?', 'Do the presidential candidates actually offer a range of promises and slogans to attract voters?']}, 'The area will be urbanized and developed, providing citizens with leisure activities and sports options': {'Questions': ["What is the claim referring to with 'The area will be urbanized and developed'?", 'What kind of leisure activities and sports options are 

{'The city of Veles would not be what it is without these market vendors.': {'Questions': ['What is the relationship between the market vendors and the city of Veles?', 'How do presidential candidates in North Macedonia typically campaign for office?', 'What are the main issues or promises being made by the seven presidential aspirants competing in the election?']}, 'Citizens are disillusioned with this government and the economic catastrophe they have caused.': {'Questions': ["Is the statement 'Citizens are disillusioned with this government and the economic catastrophe they have caused' supported by evidence?", 'What specific attributes should a future President of North Macedonia possess, according to the current situation?', 'Do the presidential candidates in the 2023 election offer a range of promises and slogans that aim to attract voters?']}, 'We will need a strong battle to stabilize the economy.': {'Questions': ['What is the claim stating as a necessary condition for stabilizi

100%|██████████| 5/5 [16:33<00:00, 198.60s/it]

{'The candidate is a member of VMRO-DPMNE': {'Questions': [['The VMRO-DPMNE party has a member running for president in the election.', 'Is the candidate indeed a member of VMRO-DPMNE?', 0], ['What is the main dilemma in the presidential election in North Macedonia?', 'Can you identify what the major issue is in the electoral arena?', 1], ['Are there more than three candidates running for the presidential position in the election?', 'Count the number of aspirants mentioned and answer this question.', 2]]}, 'The candidate represents the interests of the Macedonian people': {'Questions': [['Is the claim stating that a candidate represents the interests of the Macedonian people?', 'True'], ['Does the article mention a specific date for the start of the election campaign?', 'False'], ['Are there more than 3 candidates running in the presidential election?', 'Unknown']]}, 'The candidate will work to preserve and strengthen national identity': {'Questions': ['How does the candidate plan to p

In [ ]:
read_path = "./dataset/questions-extraction/"

# Read all the claims to questions and remove the rows which are empty
for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(read_path, party + "_claims_to_questions.csv")
    pandas_df = pd.read_csv(csv_file_path)
    print("Before removing the columns for the Party " + party + " the shape is: ", pandas_df.shape)
    
    question_proofs = {}
    for index, row in pandas_df.iterrows():
        try:
            q = json.loads(row["claims_to_questions"])
            v = q.values()
            for i in v:
                for j in i.values():
                    for k in j:
                        # print(k)
                        continue
        except:
            # Remove the row from the DataFrame
            pandas_df = pandas_df.drop(index)
            # print("Removed the row with index: ", index)
            continue
                

    print("After removing the columns for the Party " + party + " the shape is: ", pandas_df.shape)
    print("\n")
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_csv_file_path = os.path.join(read_path, party + "_claims_to_questions_cleaned.csv")
    pandas_df.to_csv(cleaned_csv_file_path, index=False)
    

Before removing the columns for the Party DUI the shape is:  (112, 2)
After removing the columns for the Party DUI the shape is:  (102, 2)


Before removing the columns for the Party SDSM the shape is:  (49, 2)
After removing the columns for the Party SDSM the shape is:  (45, 2)


Before removing the columns for the Party Left the shape is:  (70, 2)
After removing the columns for the Party Left the shape is:  (60, 2)


Before removing the columns for the Party VMRO-DPMNE the shape is:  (19, 2)
After removing the columns for the Party VMRO-DPMNE the shape is:  (16, 2)




## Google Search (Evidence) Retrieval

In [21]:
def google_search(query, api_key, cse_id, num=3):
    if (type(query) == dict):
        query = query.values()
        query = str(query)
    if (type(query) == list):
        query = query[0]
    if (type(query) != str):
        query = str(query)
    print(query)
    query = quote_plus(query)
    url = f"https://www.googleapis.com/customsearch/v1?q={query}&key={api_key}&cx={cse_id}&num={num}"
    response = requests.get(url).json()
    results = []
    for item in response.get('items', []):
        # print(item)
        results.append(item.get('snippet'))
        if item.get('snippet') is None:
            print("\n")
            print("Response")
            print(response)
            print("Query")
            print(query)
    print(results)
    if (results == []):
        print("No results found")
        print(response)
    return results

In [22]:
# Evidence = {}
# Claims_and_Evidence_dict = {}
# for claim, questions in claim_to_question_dic.items():
#     print(claim)
#     print(questions)
#     for question in questions:
#         search_results = google_search(question, google_api, cx)
#         Evidence[question] = search_results
#     Claims_and_Evidence_dict[claim] = Evidence
#     Evidence = {}
    
    
    
#     # for question in questions:
#     #     search_results = google_search(question, google_api, cx)
#     #     Evidence[question] = search_results
#     # Claims_and_Evidence_dict[claim] = Evidence
#     # Evidence = {}
#     # print(Claims_and_Evidence_dict)


read_questions_path = "./dataset/questions-extraction/"
save_question_path = "./dataset/proof-extraction/"
# Extract Proofs from Google
for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(read_questions_path, party + "_claims_to_questions_cleaned.csv")
    pandas_df = pd.read_csv(csv_file_path)
    
    
    
    for index, row in tqdm.tqdm(pandas_df.iterrows(), total=pandas_df.shape[0]):
        # try:
        id = row['ad_archive_id']
        rowJson = json.loads(row["claims_to_questions"])
        for key, value in rowJson.items():
            questions = value.get("Questions", [])
            enriched_questions = []
            
            for question in questions:
                search_results = google_search(question, google_api, cx)
                enriched_questions.append({
                    'question': question,
                    'search_results': search_results
                })
                time.sleep(0.6)
                
            rowJson[key]['Questions'] = enriched_questions
            # Update the DataFrame with the enriched questions
            pandas_df.at[index, 'claims_to_questions'] = json.dumps(rowJson)


            
        # except:
        #     # Remove the row from the DataFrame
        #     pandas_df = pandas_df.drop(index)
        #     print("Removed the row with index: ", index)
        #     # print("Removed the row with index: ", index)
    
        
    # Save the cleaned DataFrame to a new CSV file
    cleaned_csv_file_path = os.path.join(save_question_path, party + "_claims_to_questions_to_proofs.csv")
    pandas_df.to_csv(cleaned_csv_file_path, index=False)
    print(pandas_df.head())
    

  0%|          | 0/102 [00:00<?, ?it/s]

What are the key attributes that the next President of North Macedonia should possess?
['Jun 3, 2022 ... It is often said that the North Atlantic Treaty Organization was founded in response to the threat posed by the Soviet Union.', 'As a result, key local services, such as public transportation, have been severely impacted. ... Inflation would likely have been much higher if the\xa0...', 'Rapidly reducing methane emissions from energy, agriculture, and waste can achieve near-term gains in our efforts in this decade for decisive action and is\xa0...']
How do the presidential candidates differ in their promises and slogans?
['In those days, the candidate receiving the second-highest vote became the vice president. In a scheme to deny Adams the presidency, Alexander Hamilton\xa0...', 'Jan 11, 1989 ... Well, back in 1980, when I was running for President, it was all so different. Some pundits said our programs would result in catastrophe. Our\xa0...', 'The Campaign and Election of 1912. A

  1%|          | 1/102 [00:07<13:09,  7.81s/it]

Is the claim 'SDSM is the only party ready to fight for citizens and the future of the state' supported by factual evidence?
['state any child support they get in the future. The ... REBUTTAL - Evidence presented at trial by one party in order to overcome evidence introduced by.', 'Jun 24, 2024 ... Title II of the ADA - regulations for state and local governments meant to ensure that people with disabilities can participate in all\xa0...', 'Jan 31, 2024 ... A large body of research indicates that fact-checking can be an effective way to correct false beliefs about specific claims, especially for\xa0...']
What specific attributes should the next President of North Macedonia possess, according to the author Enis Shaqiri?
['May 19, 2022 ... Petrović, in North Macedonia, another new locality was recognized ... new technology will be built have been laid. Many innovations have.', 'May 17, 2024 ... *Corresponding author e-mail:emri.murati@unite.edu.mk. Abstract. In the Republic of Northern M

  2%|▏         | 2/102 [00:15<12:49,  7.69s/it]

...

What is the relationship between the market vendors and the city of Veles?
['... city market Veles” to be treated with the financial sources from the ... The Project for construction of a city green market area, envisages also connections to\xa0...', 'Aug 10, 2021 ... As the title implies, these quests bring you higher in the tiered city of Veles as you take on more important gigs. You gather new armor\xa0...', "Welcome to Baltimore City's first Annual Equity Report. In September of 2018, Baltimore City Council passed the Equity. Assessment Ordinance, which became\xa0..."]
How do presidential candidates in North Macedonia typically campaign for office?
['... usually determine who the final presidential nominee ... The presidential candidates campaign throughout the country to win the support of the general population.', 'Apr 27, 2014 ... ... campaign silence for the presidential election. ... predominantly by men. 21. However, the number of instances of damage to campaign offices.', "A

 94%|█████████▍| 15/16 [02:00<00:08,  8.61s/it]

The VMRO-DPMNE party has a member running for president in the election.
['Apr 25, 2024 ... Presidential candidate of the biggest opposition party VMRO-DPMNE Gordana Siljanovska-Davkova celebrates her Gordana Siljanovska-Davkova\xa0...', 'Apr 27, 2014 ... While all contestants were covered largely in a neutral manner, the VMRO-DPMNE had ... purchased by VMRO-DPMNE and their presidential candidate;\xa0...', 'Apr 24, 2024 ... The second woman candidate in the presidential race is Biljana ... VMRO-DPMNE is committed to continuing EU accession talks, but they\xa0...']
What is the main dilemma in the presidential election in North Macedonia?
["Mar 6, 2024 ... Eyeing Victory, North Macedonia's Opposition Faces Post-Election Dilemma. The presidential candidate of the biggest opposition VMRO DPMNE\xa0...", 'Apr 3, 2024 ... The 2024 presidential elections in North Macedonia are shaping up to ... election is shaping up to be particularly challenging for the main\xa0...', 'Apr 15, 2019 ... On Apr

100%|██████████| 16/16 [02:09<00:00,  8.07s/it]

      ad_archive_id                                claims_to_questions
0   427480589881211  {"The anti-Albanian quarters have a tendency t...
1  1636678057070325  {"Macedonia is yours again!": {"Questions": [{...
2  1463429004244655  {"There is no need for a president like Pendar...
3   392857170323758  {"Vote for number 2 on May 8th": {"Questions":...
4   403410729140623  {"There is only one truth": {"Questions": [{"q...


## Identify Stance of Evidence

In [23]:
def detect_stance(claim, evidence):
    prompt = f"""Analyze whether this evidence supports or refutes the claim.
    Claim: "{claim}"
    Evidence: "{evidence}"
    Respond only with on of the following options: Completely Supports, Partially Support, Refute, Irrelevant, or No Evidence.
    """
    extract_stance_payload = {
    "model": "llama3",
    "prompt": prompt,
    }
    stance_response = requests.post(llama3_url, json=extract_stance_payload)
    if stance_response.status_code == 200:
        # Process the streamed response
        lines = stance_response.text.splitlines()
        output = ""

        for line in lines:
            try:
                # Parse each line as JSON
                line_data = json.loads(line)
                # Concatenate the "response" field
                output += line_data.get("response", "")
            except json.JSONDecodeError:
                # Skip lines that are not valid JSON
                continue

        # Parse the final concatenated output as JSON
        # Remove everything after the newline character
        output = output.split('\n')[0]
        return output
    else:
        print(f"Error: {stance_response.status_code}")
        print(stance_response.text)
    return None

In [25]:
save_file_path = "./dataset/stance-extraction/"
read_file_path = "./dataset/proof-extraction/"

for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(read_file_path, party + "_claims_to_questions_to_proofs.csv")
    pandas_df = pd.read_csv(csv_file_path)
    
    # Print the DataFrame shape
    print(f"Shape of {party} DataFrame:", pandas_df.shape)
    
    # Check if the DataFrame is empty
    if pandas_df.empty:
        print(f"The DataFrame for {party} is empty.")
        continue
    
    # Iterate through the rows of the DataFrame
    for index, row in tqdm.tqdm(pandas_df.iterrows(), total=pandas_df.shape[0]):
        # Extract the text from the DataFrame
        id = row['ad_archive_id']
        rowJson = json.loads(row["claims_to_questions"])
        for key, value in rowJson.items():
            questions = value.get("Questions", [])
            enriched_questions = []
            
            for question in questions:
                stance = detect_stance(key, question['search_results'])
                question['stance'] = stance
                enriched_questions.append(question)
                
            rowJson[key]['Questions'] = enriched_questions
            # Update the DataFrame with the enriched questions
            pandas_df.at[index, 'claims_to_questions'] = json.dumps(rowJson)
            print(stance)

            
        # Save the cleaned DataFrame to a new CSV file
        cleaned_csv_file_path = os.path.join(save_file_path, party + "_claims_to_questions_to_proofs_with_stance.csv")
        pandas_df.to_csv(cleaned_csv_file_path, index=False)

Shape of DUI DataFrame: (102, 2)


  0%|          | 0/102 [00:00<?, ?it/s]

No Evidence. The evidence provided appears to be discussing past events and politicians, including Donald Trump's potential return to the White House and historical presidential elections. It does not provide any information that directly supports or refutes the claim "НемаОткажуваќьOdИднината", which seems to be a Bulgarian phrase without an apparent meaning in English.
Partially Supports


  1%|          | 1/102 [00:07<12:12,  7.25s/it]

Completely Supports
No Evidence.
Refute


...

No Evidence.
Refute
Refute


 94%|█████████▍| 15/16 [01:26<00:05,  5.71s/it]

Irrelevant
Irrelevant
Irrelevant


100%|██████████| 16/16 [01:30<00:00,  5.64s/it]

No Evidence.


In [37]:
read_file_path = "./dataset/stance-extraction/"

# Read all the stances
for party in parties:
    # Read the CSV file into a DataFrame
    csv_file_path = os.path.join(read_file_path, party + "_claims_to_questions_to_proofs_with_stance.csv")
    pandas_df = pd.read_csv(csv_file_path)
    
    # Reading the stances
    for index, row in pandas_df.iterrows():

        q = json.loads(row["claims_to_questions"])
        v = q.values()
        for i in v:
            for j in i.values():
                for k in j:
                    stance = k["stance"]
                    if "Irrelevant" in stance:
                        k["stance"] = "Irrelevant"
                    elif "No Evidence" in stance:
                        k["stance"] = "No Evidence"
                    elif "Completely Supports" in stance:
                        k["stance"] = "Completely Supports"
                    elif "Partially Support" in stance:
                        k["stance"] = "Partially Supports"
                    elif "Refute" in stance:
                        k["stance"] = "Refute"
                        
                    # Update the DataFrame with the enriched questions
                    pandas_df.at[index, 'claims_to_questions'] = json.dumps(q)
                    # Save the cleaned DataFrame to a new CSV file
                    cleaned_csv_file_path = os.path.join(read_file_path, party + "_claims_to_questions_to_proofs_with_stance.csv")
                    pandas_df.to_csv(cleaned_csv_file_path, index=False)
                    print(k["stance"])
                    continue

Irrelevant
Refute
No Evidence
Refute
Partially Supports
Partially Supports
Partially Supports
Partially Supports
Completely Supports
Refute
No Evidence
No Evidence
Partially Supports
Irrelevant
Refute
Refute
Irrelevant
Refute
Irrelevant
No Evidence
Refute
Refute
Refute
Refute
Irrelevant
Refute
No Evidence
Completely Supports
Irrelevant
Irrelevant
Partially Supports
Irrelevant
Refute
Partially Supports
Partially Supports
Refute
Partially Supports
Irrelevant
Irrelevant
Refute
Partially Supports
No Evidence
No Evidence
Partially Supports
Irrelevant
Refute
Refute
Irrelevant
Refute
No Evidence
No Evidence
Partially Supports
No Evidence
Refute
Partially Supports
Partially Supports
Irrelevant
Partially Supports
No Evidence
Refute
Partially Supports
Irrelevant
Irrelevant
Partially Supports
Partially Supports
Partially Supports
No Evidence
Irrelevant
Irrelevant
Irrelevant
Irrelevant
Partially Supports
No Evidence
No Evidence
Partially Supports
Partially Supports
Refute
Refute
Refute
Refute
Refu

## Determine Correction

In [ ]:
def determine_verdict(stance_results):
    verdicts = []
    for claim, questions in stance_results.items():
        for question, evidences in questions.items():
            # Make sure that all the stance results are in the same format
            for evidence in evidences:
                for key in evidence:
                    if key == 'stance':
                        stance = evidence['stance']
                        
                        if stance != 'Completely Supports' and stance != 'Partially Support' and stance != 'Refute':
                            evidence['stance'] = 'Irrelevant'
                            # Search if the sentence contains any of the words
                            if 'Completely Supports' in stance:
                                evidence['stance'] = 'Completely Supports'
                            elif 'Partially Support' in stance:
                                evidence['stance'] = 'Partially Support'
                            elif 'Refute' in stance:
                                evidence['stance'] = 'Refute'
                            elif 'Irrelevant' in stance:
                                evidence['stance'] = 'Irrelevant'
                            elif 'No Evidence' in stance:
                                evidence['stance'] = 'No Evidence'
                        print(stance)
                    
            
            supports = sum(1 for evidence in evidences if evidence['stance'] == 'Completely Supports' or evidence['stance'] == 'Partially Support')
            refutes = sum(1 for evidence in evidences if evidence['stance'] == 'Refute')
            if supports > refutes:
                verdicts.append((claim, question, "Supports"))
            elif refutes > supports:
                verdicts.append((claim, question, "Refutes"))
            else:
                verdicts.append((claim, question, "Neutral"))
                
            
            
    return verdicts

In [124]:
determine_verdict(stance_results)

Partially Support
Partially Support
Refute
Partially Support
Partially Support
Partially Support
Partially Support
No Evidence
Irrelevant
Refute
Refute
Refute
Refute
Refute
Refute
Refute
Refute
Partially Support
Partially Support
Partially Support
No Evidence
Partially Support
Partially Support
Partially Support
Partially Support
Irrelevant
Refute
No Evidence
Partially Support
Partially Support
Irrelevant
Irrelevant
Irrelevant
Partially Support
Partially Support
Partially Support
Irrelevant
Partially Support
Partially Support
Partially Support
Partially Support
Partially Support
No Evidence
Partially Support
Partially Support
No Evidence
Partially Support
No Evidence
No Evidence
Partially Support
Partially Support
No Evidence
Partially Support
Partially Support
No Evidence
Irrelevant
Irrelevant


[('Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren',
  "What is the purpose of singing national anthems according to Hristijan Mickoski's proposal?",
  'Supports'),
 ('Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren',
  'Is it true that VMRO-DPMNE promotes nationalism and anti-European agendas, as claimed by the author?',
  'Supports'),
 ('Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren',
  'Does the author suggest that VMRO-DPMNE isolates the country and hinders its future prospects in Europe?',
  'Supports'),
 ('The proposal of the leader of VMRO-DPMNE Hristijan Mickoski to make children sing the national anthem every month reflects the nationalist mindset of VMRO-DPMNE',
  'Is the proposal to make children sing the national anthem every month a common practice in any country?',
  'Refutes'),
 ('The proposal of the leader of VMRO-DPMNE Hristijan Mickoski to make children sing the national anthem every month re

In [125]:
def claims_verdict(claims, stance_results):
    verdicts = []
    for claim in claims:
        supports = sum(1 for question, evidences in stance_results[claim].items() if any(evidence['stance'] == 'Completely Supports' or evidence['stance'] == 'Partially Support' for evidence in evidences))
        refutes = sum(1 for question, evidences in stance_results[claim].items() if any(evidence['stance'] == 'Refute' for evidence in evidences))
        if supports > refutes:
            verdicts.append((claim, "Supports"))
        elif refutes > supports:
            verdicts.append((claim, "Refutes"))
        else:
            verdicts.append((claim, "Neutral"))
    return verdicts
claims_verdict(claims, stance_results)

[('Himni dhe simbolet shtetërore kanë vendin dhe kohën e vet se ku përdoren',
  'Supports'),
 ('The proposal of the leader of VMRO-DPMNE Hristijan Mickoski to make children sing the national anthem every month reflects the nationalist mindset of VMRO-DPMNE',
  'Refutes'),
 ('VMRO-DPMNE spreads unjust nationalism while making excuses for anti-European agendas',
  'Supports'),
 ('VMRO-DPMNE fosters hatred and division', 'Supports'),
 ('VMRO-DPMNE blocks the European future of young generations', 'Supports'),
 ('VMRO-DPMNE isolates the country', 'Supports'),
 ('VMRO-DPMNE exercises power against Albanian nation in the municipalities that lead its quotas',
  'Supports')]